This Notebook will be used for Week 4 Coding Purpose 

Project Title - (Data Anaysis for Real Estate Development Work in Patna)

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from geopy.geocoders import Nominatim
import geopy 
import geocoder
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

print('Import Successful')

Import Successful


In [2]:
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Patna')
data = data.text

In [3]:
soup = BeautifulSoup(data, 'html.parser')

In [4]:
neighbourhoodList = []

In [5]:
for row in soup.find_all('div', class_='mw-category')[0].find_all('li'):
    neighbourhoodList.append(row.text)

In [6]:
patna_df = pd.DataFrame({'Neighbourhood' : neighbourhoodList})

patna_df.head()

,Neighbourhood
0,Ashok Rajpath
1,Badalpura
2,"Bailey Road, Patna"
3,Bakhtiarpur
4,Bankipore


In [7]:
patna_df.shape

(41, 1)

Getting Geographical Coordinates

In [8]:
def get_latlng(Neighbourhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},Patna,Bihar'.format(Neighbourhood))
        lat_lng_coords = g.latlng

    return lat_lng_coords

In [9]:
coords = [get_latlng(Neighbourhood) for Neighbourhood in patna_df['Neighbourhood'].tolist()]

In [10]:
coords

[[25.623740000000055, 85.12997000000007],
 [25.57955000000004, 85.14967000000007],
 [25.60978088589063, 85.12355592308616],
 [25.46064000000007, 85.53914000000003],
 [25.61886000000004, 85.15568000000007],
 [25.46497000000005, 85.70828000000006],
 [25.606160000000045, 85.12250000000006],
 [25.600348323046713, 85.17543527923095],
 [25.56194000000005, 84.86305000000004],
 [25.62570995343536, 85.11039552862357],
 [25.62101000000007, 85.12653000000006],
 [25.610940021281564, 85.13090028193753],
 [25.621120000000076, 85.04452000000003],
 [25.640540000000044, 85.09963000000005],
 [25.636960000000045, 85.01285000000007],
 [25.608480000000043, 85.14199000000008],
 [25.508150000000057, 85.30690000000004],
 [25.612320000000068, 85.13880000000006],
 [25.61230000000006, 85.20228000000003],
 [25.616460000000075, 85.14851000000004],
 [25.606190000000026, 85.20052000000004],
 [25.582190000000026, 85.12629000000004],
 [25.592550000000074, 85.14625000000007],
 [25.58418000000006, 85.04353000000003],
 [

In [11]:
df_coords = pd.DataFrame(coords, columns = ['Latitude','Longitude'])

In [12]:
patna_df['Latitude'] = df_coords['Latitude']
patna_df['Longitude'] = df_coords['Longitude']

In [13]:
print(patna_df.shape)

(41, 3)


In [14]:
patna_df

,Neighbourhood,Latitude,Longitude
0,Ashok Rajpath,25.623740,85.129970
1,Badalpura,25.579550,85.149670
2,"Bailey Road, Patna",25.609781,85.123556
3,Bakhtiarpur,25.460640,85.539140
4,Bankipore,25.618860,85.155680
5,Barh,25.464970,85.708280
6,Beer Chand Patel Path,25.606160,85.122500
7,Bhootnath Road,25.600348,85.175435
8,Bihta,25.561940,84.863050
9,Boring Road,25.625710,85.110396


In [15]:
patna_df.to_csv('patna_df.csv', index = False)

Creating Map of Patna with its Neighbourhood on top

In [16]:
address = 'Patna, Bihar'

geolocator = Nominatim(user_agent='Patna')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geographical Coordinates of Patna {}, {}'.format(latitude, longitude))

Geographical Coordinates of Patna 25.6093239, 85.1235252


In [17]:
map_patna = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lng, Neighbourhood in zip(patna_df['Latitude'], patna_df['Longitude'], patna_df['Neighbourhood']):
    label = '{}'.format(Neighbourhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng], radius = 5, popup = label, color = 'green',
        fill = True, fill_color = '#3186cc', fill_opacity = 0.7
    ).add_to(map_patna)

map_patna

In [18]:
map_patna.save('map_patna.html')

Using Foursquare API to explore the neighbourhood i.e. to get the top 100 venues that are within a radius of 2000 meters

In [19]:
CLIENT_ID = 'ZB1TDQHGNJBFB2SWNLEO1X2XBFSSRJCMYZLQX4GO4AINNH2Z'
CLIENT_SECRET = 'XG3J3WBJOVUXD0KEOVUBFKFISILUJTEBL5EVZHR2FX1EWJEL'
VERSION = '20200703'


In [38]:
radius = 10000
LIMIT = 100

venues = []

for lat, lng, Neighbourhood in zip(patna_df['Latitude'], patna_df['Longitude'], patna_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID, CLIENT_SECRET, VERSION,
        lat, lng, radius, LIMIT
    )

results = requests.get(url).json()['response']['groups'][0]['items']

for venue in results:
    venues.append((
        Neighbourhood, lat, lng,
        venue['venue']['name'],
        venue['venue']['location']['lat'],
        venue['venue']['location']['lng'],
        venue['venue']['categories'][0]['name']
    ))

In [39]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(24, 7)


,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Shivpuri, Patna",25.58239,85.10094,Patna Golf Club,25.602367,85.095892,Golf Course
1,"Shivpuri, Patna",25.58239,85.10094,Patna Zoo,25.600763,85.097267,Park
2,"Shivpuri, Patna",25.58239,85.10094,Cafe Coffee Day,25.620594,85.112876,Café
3,"Shivpuri, Patna",25.58239,85.10094,17 Degrees,25.634401,85.106257,Indian Restaurant
4,"Shivpuri, Patna",25.58239,85.10094,Gandhi Maidan,25.617395,85.145020,Park


In [40]:
venues_df.groupby(['Neighbourhood']).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhood,,,,,,
"Shivpuri, Patna",24,24,24,24,24,24


In [41]:
print('No of unique categories {}'.format(len(venues_df['VenueCategory'].unique())))

No of unique categories 16


In [42]:
venues_df['VenueCategory'].unique()[:9]

array(['Golf Course', 'Park', 'Café', 'Indian Restaurant',
       'Shopping Mall', 'Hotel Bar', 'Fast Food Restaurant',
       'Chinese Restaurant', 'Pizza Place'], dtype=object)

In [44]:
'Neighborhood' in venues_df['VenueCategory'].unique()

False

Analyze Each Neighbourhood

In [45]:
patna_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix = "", prefix_sep = "")

patna_onehot['Neighbourhood'] = venues_df['Neighbourhood']

fixed_columns = [patna_onehot.columns[-1]]+list(patna_onehot.columns[:-1])
patna_onehot = patna_onehot[fixed_columns]

print(patna_onehot.shape)
patna_onehot.head()

(24, 17)


,Neighbourhood,American Restaurant,Business Service,Café,Chinese Restaurant,Department Store,Fast Food Restaurant,Golf Course,Hotel,Hotel Bar,Indian Restaurant,Juice Bar,Multiplex,Park,Pizza Place,Shopping Mall,Train Station
0,"Shivpuri, Patna",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,"Shivpuri, Patna",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,"Shivpuri, Patna",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Shivpuri, Patna",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,"Shivpuri, Patna",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [46]:
patna_grouped = patna_onehot.groupby(['Neighbourhood']).mean().reset_index()

print(patna_grouped.shape)
patna_grouped

(1, 17)


,Neighbourhood,American Restaurant,Business Service,Café,Chinese Restaurant,Department Store,Fast Food Restaurant,Golf Course,Hotel,Hotel Bar,Indian Restaurant,Juice Bar,Multiplex,Park,Pizza Place,Shopping Mall,Train Station
0,"Shivpuri, Patna",0.041667,0.041667,0.125,0.041667,0.041667,0.083333,0.041667,0.083333,0.041667,0.083333,0.041667,0.041667,0.083333,0.125,0.041667,0.041667


Observation


Due to Lack Of data for the coordinates of patna on Foursquare API we cannot analyze all the parts of patna.
there was only one location for which there was data present 
this location is called Shivpuri, Patna.

lack of data for a particular region is another reason for this appropriate analysis so movint further will the only availaible data, we can predict that in this region there are plenty of
Resturants, Parks, Shopping Malls.

thus, property developers are advised to prefer this location for their projects .